In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import matplotlib.pyplot as plt
from src.config import get_dataset_and_user
%matplotlib inline

In [9]:
# get data
X_iris, user_iris = get_dataset_and_user('iris')
y_iris = user_iris.get_label(X_iris, update_counter=False)
X_iris['dummy'] = 1

X_housing, user_housing = get_dataset_and_user('housing')
y_housing = user_housing.get_label(X_housing, update_counter=False)
X_housing['dummy'] = 1

In [12]:
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
clf = LinearSVC(C=1000, fit_intercept=True)
clf.fit(X_iris,y_iris)
f1_score(clf.predict(X_iris), y_iris)

1.0

In [13]:
from src.active_learning.svm import SimpleMargin, SolverMethod, OptimalMargin
from src.active_learning.agnostic import RandomLearner
from src.initial_sampling import FixedSizeStratifiedSampler
from src.showdown import Showdown
from src.plotting import plot_showdown

user_housing.max_iter = 20
user_iris.max_iter = 10

active_learners_list = [
     ("random", RandomLearner(LinearSVC(C=1000, fit_intercept=False))),
     ("simple margin", SimpleMargin(kind='linear', C=1000, fit_intercept=True)),
     ('optimal margin', OptimalMargin(C=1000, fit_intercept=True))
     #("rbfSVM", SimpleMargin(C=1000, kernel='rbf')),
     #("solver5", SolverMethod(pool_size=5, C=1000, kernel='linear')),
     #("solver10", SolverMethod(pool_size=10, C=1000, kernel='linear')),
     #("solver20", SolverMethod(pool_size=20, C=1000, kernel='linear'))
     #("actboost", ActBoost(chain_length=100, sample_size=16, n_iterations=500)),
     #("random", RandomLearner(Adaboost(n_iterations=500))),
     #("qbb", QueryByBoosting(n_iterations=500)), 
]

datasets_list = [
    ("housing", X_housing, user_housing),
    #("iris", X_iris, user_iris),
    #("polygon1", X_pol, user_pol),
    #("mushroom", X_mush, user_mush),
    #("dermatology", X_derm, user_derm)
]

times = 5
initial_sampler = FixedSizeStratifiedSampler(sample_size=2)
showdown = Showdown(times, initial_sampler)
output = showdown.run(datasets_list, active_learners_list)

#print(output.head())
plot_showdown(output, times, metrics_list=['fscore', 'version_space_ratio_estimate'])

[[ -8.92768840e-01   1.10687211e+00   1.40174259e+00   1.65349470e+00
   -1.06142678e+00  -1.72676157e-01   1.03031436e+00]
 [  7.16311625e-01  -7.38200426e-01  -2.10538413e-01  -5.83174182e-01
    1.06142678e+00   1.72676157e-01  -1.03031436e+00]
 [  7.50759782e-01  -6.49647342e-01  -2.10538413e-01  -5.50795805e-01
    1.06142678e+00   1.72676157e-01  -1.03031436e+00]
 [ -3.66175854e-01  -4.69232402e-01  -2.10538413e-01  -5.36263243e-01
   -1.12637914e-01  -2.30183934e-01   7.31045296e-01]
 [ -1.15203781e+00  -2.60550314e-01  -2.10538413e-01   2.42230550e-01
   -1.04692884e+00  -3.11835829e-01   8.53358138e-01]
 [ -8.71323957e-01   9.21981518e-01  -2.10538413e-01   1.34725681e+00
   -1.70514291e-01  -2.20481574e-01   2.56706790e-01]
 [ -3.85978192e-01  -4.12424500e-01  -2.10538413e-01  -1.23217663e-01
   -1.70514291e-01  -2.20481574e-01   2.56706790e-01]
 [  6.36036530e-01  -5.73573446e-01  -2.10538413e-01  -9.56140428e-02
    1.06142678e+00   1.72676157e-01  -1.03031436e+00]
 [  2.05

RuntimeError: Point outside search space!
Inequality constrain: False
Ball constrain: True


## Next steps
    - add logging
    - label points prematurely based on whether they cut the current version space or not (cut estimate = # samples)
    - add offset (add column of 1's ?) -> works on above cases. Should test on sdss also.
    - change classifier to average of sampled w's (similar to majority vote in ALuMa)
    - add sdss queries